## TODO

- [ ] Upload the required files online (ChIP-nexus and ChIP-seq)
  - use different subsets of the data
- [ ] Write the download scripts
- [ ] Write the dataspecs
- [ ] Run model training with existing `gt ...` and the existing gin config files for each.
  - document the required steps
- [ ] Implement the new interface `bpnet train` supporting the premade gin files etc.
- [ ] Replace the old commands with new commands
- [ ] Implement the end-to-end tests for these examples
- [ ] Write a brief overview to the main README and point to the examples.

## I want to be able to

- Specify my own bigwigs together with control experiments
- Use the same control experiment for multiple tracks
- Train only in peak regions or genome-wide
- Train on AWS and in Colab notebook
- Fully utilize my GPU without waiting for data-loading
- Easily try out multiple hyper-parameters
- See all the logs and evaluation metrics in the file
- See all the logs on comet.ml or similar website
- Visualize the loss curves and see some example predictions (observed vs predicted) afterwards
- Use my own evaluation notebook
- Have a good set of default hyper-parameters for each assay (ChIP-nexus and ChIP-seq)
- Train on multiple assays simultaneously
- Train on any functional genomics assay where events result in coverage peaks
  - ChIP-nexus
  - ChIP-seq
  - DNase
  - eClip
  - CutNRun
- Specify my own architecture, loss function, training function if needed

## Implementation

- Load the data into memory if possible
  - memory restriction
- Use gin-config files for specifying architectures, loss functions etc
  - Note: You have to educate users about it. It may be too complicated.
- Use SeqModel
- Use the default model without specifying any hyper-parameters for different kinds of data
- Infer the number of tasks from dataspec

## Case studies / examples

Support training the model on the following tracks:

- ChIP-nexus
  - default hyper-parameters
  - fewer layers
  - peaks and genome-wide
  - poisson loss function (don't split)
- ChIP-seq
  - bpnet9-chipseq + fewer layers
- DNase-seq
  - just mock the DNase coverage profile by adding the two bigwigs together
  - note that this is hypothetical
  - single output task
- ChIP-nexus, ChIP-seq and DNase
  - use two differnet output heads potentially. Have differnet types of heads for different tasks.
  
For each case study, have a directory with different config files + README on what to run. For example `examples/ChIP-nexus/default` or `examples/ChIP-nexus/poisson-loss`.

### Test

- Use these case studies in the unit-tests - train only for a single epoch... Test the output files. Compute the importance scores on a few regions.
- Test TF modisco input validation from all the 4 contrib score files.
- Run the full tf modisco pipeline on only a single ImpScoreFile.

## TODO

- specify only a single track
  - do we really need to specify pos_track and neg_track explicitly or can we just list them? 

# Training BPNet on your own data

<img src="figs/bpnet-arch.png" alt="BPNet" style="width: 450px;"/>

## 1. Specify data -> write `dataspec.yml`

The first step requires specifying the data on which to train the model. BPNet takes as input nucleotide sequence and outputs the read coverage profile for multiple tracks at base-resolution. The coverage tracks can come from any genome-wide functional genomics assay that has a sufficient spatial resolution including ChIP-nexus, ChIP-exo, ChIP-seq, DNase-seq, and ATAC-seq. Additionally, different experiments may have differnet biases that need to be accounted for. Both, the signal and the bias/control tracks have to be stored in [BigWig](https://genome.ucsc.edu/goldenpath/help/bigWig.html) files.

In this tutorial, we will use the data from the BPNet paper (TODO - link) measuring TF binding of 4 TFs (Oct4, Sox2, Nanog and Klf4) with ChIP-nexus in mouse embryonic stem cells (mESCs). To make things faster, we will focus only on a small subset of the regions.

- [ ] describe `dataspec.yml`
  - give one example
  - explain each entry in it

First, download the required data

In [4]:
#!wget ....
#!

`dataspec.yaml` for these data looks as follows

```yaml
fasta_file: ./reference-genome.fa
task_specs:
  task1:
    pos_counts: ./task1.pos.bigWig
    neg_counts: ./task1.neg.bigWig
    peaks: ./task1.peaks.bed.gz  # optional. Peaks associated with task1
    
    # optional
    ignore_strand: False  # if True, use the sum of pos_ and neg_counts
    bias_bigwig: ./InputDNA.counts.bigWig  # measured bias. IP in ChIP-seq
    bias_model: null   # sequence-based bias model
  task2:
    ...  # similarly as for task1
  task3:
    ...
```

In [10]:
!cat dataspec.yml

fasta_file: data/mm10_no_alt_analysis_set_ENCODE.fasta
task_specs:
  Oct4:
    pos_counts: data/Oct4/counts.pos.bw
    neg_counts: data/Oct4/counts.neg.bw
    peaks: data/Oct4/idr-optimal-set.summit.bed.gz
  Sox2:
    pos_counts: data/Sox2/counts.pos.bw
    neg_counts: data/Sox2/counts.neg.bw
    peaks: data/Sox2/idr-optimal-set.summit.bed.gz
  Nanog:
    pos_counts: data/Nanog/counts.pos.bw
    neg_counts: data/Nanog/counts.neg.bw
    peaks: data/Nanog/idr-optimal-set.summit.bed.gz
  Klf4:
    pos_counts: data/Klf4/counts.pos.bw
    neg_counts: data/Klf4/counts.neg.bw
    peaks: data/Klf4/idr-optimal-set.summit.bed.gz

bias_specs:
  input:
    pos_counts: data/patchcap/counts.pos.bw
    neg_counts: data/patchcap/counts.neg.bw
    tasks:
      - Oct4
      - Sox2
      - Nanog
      - Klf4

The `dataspec.yml` file contains three parts:
- `task_specs`
- `bias_specs`
- `fasta_file`.

### How can I visualize the raw data before training the model?

Glad you asked. Before you jump ahead and start training the model, we recommend eyeballing the coverage tracks (BigWig) and peak regions (bed) using the genome browser such as the [WashU](https://epigenomegateway.wustl.edu/) or [IGV](https://software.broadinstitute.org/software/igv/). If you can not identify peaks by eye then the model will not be able to do it either.

Having specified your data in `dataspec.yml`, you can use also `bpnet.specs.DataSpec` to parse the file and visualize the tracks for a specific genomic interval in your jupyter notebook.

In [5]:
## TODO - show how to do that. Show the Oct4 enhancer from the paper

### How do I get my data into a BigWig file?

Functional genomics experiments based on sequencing yield many short reads which then get aligned to the reference genome. The alignment locations of the reads are typically stored in the [BAM](http://samtools.github.io/hts-specs/SAMv1.pdf) file. There are different ways of computing the coverage tracks from aligned reads. To prevent loosing any spatial information in the profiles, we would like to generate non-smoothed tracks (as raw as possible). For ChIP-exo/nexus/seq experiments this means counting the 5' locations of the reads. Note that the aligned reads also have strand information hence we will generate two coverage tracks, one for the positive/forward and one for the negative/reverse strand. If multiple technical or biological replicate experiments were performed for a specific transcription factor, we will simply add up their coverage tracks.

The `bpnet` CLI offers a simple convenience command to convert the read alignments stored in the BAM file into the coverage tracks.

```bash
bpnet align2bigwig <rep1.bam> <rep2.bam> --fragment-point=5prime --strand-spec <output prefix>
```

For more information run `bpnet align2bigwig --help` or read the source code here (TODO - link). Note that `align2bigwig` also accepts the `TagAlign` file generated by the ENCODE ChIP-seq pipeline.

In [8]:
## TODO - show the read coverage profile for chip-seq and the 5' ends of the reads

### How do I get `regions.bed`?

For large genomes such as human or mouse, training genome-wide can be computationally expensive. Most of the regions in the genome will contain very little counts, hence the model will not recieve a lot of information. We can significantly speed up the training process by training the model only in regions with higher number of counts. These regions are determined using traditional peak callers such as MACS2. Since we just want to discard regions with little or no counts, we don't care about the exact peak locations or even high false positive rates. Hence almost any peak caller should be fine.

You can also train the model genome-wide by tiling the genome into bins (using say stride of 400). You can generate the intervals of the tiled genome using - TODO - refer to dataloader tools?. BPNet also allows you to sample regions overlapping a known peak with higher probability. Then you have to provide the `TODO dataloader command` with the peak locations. In that case, the produced bed file will be a tsv file looking as follows

```tsv
CHROM    START    END   task1   task2   ...
 chr1    10000  11000       1       0

```

Columns following the interval coordinates specify whether the center of the interval overlapped a peak from `task1` (1=yes) or `task2` (0=no).

### Can I train the model without the bias track?

Technically, yes. It will work well for assays with low amount of bias such as ChIP-exo or ChIP-nexus. However, we generally recommend using the bias track. 

### Can I train the model with differnet assays simultaneusly?

Yes. If you are using different assays with similar resolution (e.g. ChIP-nexus and ChIP-exo), you can just specify the bigwig files and use different bias/control files for differnt subsets of the tracks. If you are using different assays with different resolutions, you might want to tweak the parameters of model output heads to best fit the individual experiments.

**TODO - possible?**

### Should I train a single multi-task model or multiple single-task models?

If you expect the tracks to share some sequence motifs, it's likely beneficial to train a multi-task model (e.g. use a single `dataspec.yml`). Also, handling a single model is more convenient than handling multiple models. However, if you have many tracks it might be challenging to train a multi-task model.

## 2. Train the model

Having specified `dataspec.yml`, we are now ready to train the model. You can train the model with `bpnet train` as follows:

In [11]:
!bpnet train dataspec.yaml --premade=bpnet9 -o trained_model/

/bin/sh: 1: bpnet: not found


This command will train the `bpnet9` pre-made model on the `dataspec.yml` and save the trained model together with other output files to `trained_model` directory. You can view the list of available pre-made models here: [../bpnet/premade](../bpnet/premade) - **TODO create**.

Here is the overview of the generated output files:

- train.log - log file 
- model.h5 - Keras model HDF5 file
- seqmodel.pkl - Serialized SeqModel. This is the main trained model.
- eval-report.ipynb/.html - evaluation report containing training loss curves and some example model predictions. You can specify your own ipynb using `--eval-report=my-notebook.ipynb`.
- model.gin -> copied from the input
- dataspec.yaml -> copied from the input

If you would like to track your computational experiments using [wandb](https://www.wandb.com/) or [Comet.ml](https://www.comet.ml/), you can specify those by providing the project id, e.g. `--wandb=<user>/<project>`. Run `bpnet train -h` to see all the available command-line options.

## Args

- `<dataspec>`: dataspec.yaml
- `--config=model.gin` (gin config files specifying the model architecture and the loss etc)
- `--premade=bpnet9`: pre-made config file to use (e.g. use the default architecture). The user could override the setting using bindings.
- `--override`: model/loss/training parameters to override
- `--skip-evaluation` If true, the model will skip evaluation
- `--gpu, -g` Which GPU to use.
- `--memfrac-gpu` . what fraction of the GPU memory to use.
- `--eval-report`: path to the ipynb report file. Use the default one. If set to `None` or `none`, then the report will not be generated.
- `--wandb`: path to the wandb (https://www.wandb.com/) project `<username>/<project>`. 
  - This will track and upload your metrics. Make sure you have specified the following environemnt variable: TODO.
- `--cometml`: path to the comet.ml (https://www.comet.ml/) project specified as `<username>/<project>`. 
  - This will track and upload your metrics. Make sure you have specified the following environemnt variable: TODO.

## 3. Tweak the model

### Modify the hyper-parameters

The first step you can do to improve your model is to adapt the hyper-parameters of the existing model. Have a look at the default hyper-parameters of the pre-made model you were using here: [../bpnet/premade/bpnet9.gin](../bpnet/premade/bpnet9.gin). You can directly override the hyper-parameters from the command line as follows:

```bash
bpnet train dataspec.yaml --premade=bpnet9 --override='train.lr=0.05;model.multi_task_model.n_layers=5' -o trained_model/
```

This will use a different learning rate (0.05) and less convolutional layers (5) by overriding the original values. Note that multiple parameter specifications were separated using `;`. If you find it impractical to specify the hyper-parameters from the CLI, you can instead specify them in the `model.gin` config file: 

```python
train.lr = 0.05
model.multi_task_model.n_layers = 5
```

and then run `bpnet train` as follows:

```bash
bpnet train dataspec.yaml --premade=bpnet9 --config=model.gin -o trained_model/
```

Note that you can use `--override` and `--config` simultaneusly. If both specify the same parameter, then the one specified by `--override` will be used. For example, the following command will use the learning rate of 0.01.

```bash
bpnet train dataspec.yaml --premade=bpnet9 --config=model.gin --override='train.lr=0.01' -o trained_model/
```

Altogether, `--config` overrides the parameters specified by `--premade`; `--override` overrides parameters specified by `--config` and `--premade` (e.g. `--override` > `--config` > `--premade`).

### Specify your own model architecture, loss function or training procedure

Have a look at the [gin-config documentation](https://github.com/google/gin-config) to learn more about the gin config files. This will help you understand how you can utilize them effectively and thereby go beyond the premade models. To specify your own architecture, use a different loss function or a different training procedure, you have to do three things:
1. Implement a python function returning the `bpnet.seqmodel.SeqModel` object.
2. Decorate the function with `@gin.configurable`
3. Specify that you would like to use this model in the `model.gin` config file.

**TODO** provide an example where you use a poisson loss function.

#### What is `bpnet.seqmodel.SeqModel`?

`SeqModel` is a wrapper around the Keras model. It requires the input to be a one-hot-encoded DNA sequence and it consists of two key components: body and heads. Body will process the input sequence with multiple layers yielding the bottleneck activation map which can be seens as containing an embedding at each nucleotide position. Heads will take the bottleneck activation map as input and they will output the final model predictions. Heads specify the loss function and the evaluation metric. They can additionally accept the bias/control track as input and control for it in the loss function. The benefit of restricting the model architecture in that way is that the sequence contribution scores can be automatically computed with no extra code. Have a look at the `SeqModel` [source code](../bpnet/seqmodel.py) to learn more about it. 

## FAQ

### I want to train a model on ChIP-seq. How can I do this?

### I want to train a model on DNase-seq or ATAC-seq. How can I do this?

The key difference between DNase-seq and ChIP-seq/exo is that DNase-seq coverage is not strand specific. Hence a single BigWig file is required. By contrast, ChIP-seq required two BigWigs - one for the positive and one for the negative strand. Beware that controlling for DNase biases is still an open question and you should think carefully about it.

Otherwise, you can just specify a similar `dataspec.yml` as before:

```yaml
TODO: write
```